# Nextstrain Zika tutorial
Instructions from https://nextstrain.org/docs/getting-started/zika-tutorial/

In [1]:
import json

## Copy files from Nextstrain Github

In [5]:
# !git clone https://github.com/nextstrain/zika-tutorial.git

# Prepare the Sequences
## Filter the Sequences
Filter the parsed sequences and metadata to exclude strains from subsequent analysis and subsample the remaining strains to a fixed number of samples per group.

In [6]:
!mkdir -p zika-tutorial/results/

!augur filter \
  --sequences zika-tutorial/data/sequences.fasta \
  --metadata zika-tutorial/data/metadata.tsv \
  --exclude zika-tutorial/config/dropped_strains.txt \
  --output zika-tutorial/results/filtered.fasta \
  --group-by country year month \
  --sequences-per-group 20 \
  --min-date 2012


# For documentation of all `filter` commands
# !augur filter -h

### Documentation for each command
#### `!augur filter`


`--sequences zika-tutorial/data/sequences.fasta`: Sequences in fasta or VCF format

`--metadata zika-tutorial/data/metadata.tsv`: Metadata associated with sequences


`--exclude zika-tutorial/config/dropped_strains.txt`: File with list of strains that are to be excluded


`--output zika-tutorial/results/filtered.fasta`: Output file

`--group-by country year month`: Categories with respect to subsample; two virtual fields, "month" and "year", are supported if they don't already exist as real fields but a "date" field does exist.

`--sequences-per-group 20`: Subsample to no more than this number of sequences per category

`--min-date 2012`: Minimal cutoff for numerical date

## Align the Sequences
Create a multiple alignment of the sequences using a custom reference. After this alignment, columns with gaps in the reference are removed. Additionally, the --fill-gaps flag fills gaps in non-reference sequences with “N” characters. These modifications force all sequences into the same coordinate space as the reference sequence.

In [7]:
!augur align \
  --sequences zika-tutorial/results/filtered.fasta \
  --reference-sequence zika-tutorial/config/zika_outgroup.gb \
  --output zika-tutorial/results/aligned.fasta \
  --fill-gaps


using mafft to align via:
	mafft --reorder --anysymbol --thread 1 zika-tutorial/results/filtered.fasta.ref.fasta 1> zika-tutorial/results/aligned.fasta 2> zika-tutorial/results/aligned.fasta.log 

	Katoh et al, Nucleic Acid Research, vol 30, issue 14
	https://doi.org/10.1093%2Fnar%2Fgkf436

Trimmed gaps in KX369547.1 from the alignment


### Documentation for each command
#### `!augur align`

`--sequences zika-tutorial/results/filtered.fasta`

`--reference-sequence zika-tutorial/config/zika_outgroup.gb`: Strip insertions relative to reference sequence; use if the reference is NOT already in the input sequences

`--output zika-tutorial/results/aligned.fasta`

`--fill-gaps`: if gaps represent missing data rather than true indels, replace by N after aligning

**We'll want this to be off. We expect that indels are true, not missing data**

# Construct the Phylogeny
Infer a phylogenetic tree from the multiple sequence alignment.

In [8]:
!augur tree \
  --alignment zika-tutorial/results/aligned.fasta \
  --output zika-tutorial/results/tree_raw.nwk

Building a tree via:
	iqtree -ninit 2 -n 2 -me 0.05 -nt 1 -s zika-tutorial/results/aligned-delim.fasta -m GTR > zika-tutorial/results/aligned-delim.iqtree.log
	Nguyen et al: IQ-TREE: A fast and effective stochastic algorithm for estimating maximum likelihood phylogenies.
	Mol. Biol. Evol., 32:268-274. https://doi.org/10.1093/molbev/msu300

Building original tree took 0.42250800132751465 seconds


### Documentation for each command
#### `!augur tree`

`!augur tree` can use [FastTree](http://www.microbesonline.org/fasttree/), [RAxML](https://sco.h-its.org/exelixis/web/software/raxml/index.html), or [IQ-TREE](http://www.iqtree.org/) (default)

`--alignment zika-tutorial/results/aligned.fasta`: Alignment in fasta or VCF format

`--output zika-tutorial/results/tree_raw.nwk`

#### Print tree_raw

In [89]:
from ete3 import Tree


tree_raw = Tree("zika-tutorial/results/tree_raw.nwk")
print(tree_raw)


   /-PAN/CDC_259359_V1_V3/2015
  |
  |      /-COL/FLR_00024/2015
  |   /-|
  |--|   \-COL/FLR_00008/2015
  |  |
  |   \-VEN/UF_1/2016
  |
  |   /-Colombia/2016/ZC204Se
  |  |
  |  |      /-Brazil/2015/ZBRC301
  |  |     |
  |  |     |            /-Brazil/2015/ZBRA105
--|  |     |           |
  |  |     |           |               /-USA/2016/FL022
  |  |     |           |            /-|
  |  |     |           |         /-|   \-USA/2016/FLWB042
  |  |     |           |        |  |
  |  |     |         /-|      /-|   \-DOM/2016/BB_0059
  |  |     |        |  |     |  |
  |  |     |        |  |     |  |   /-USA/2016/FLUR022
  |  |     |        |  |   /-|   \-|
  |  |     |        |  |  |  |      \-Aedes_aegypti/USA/2016/FL05
  |  |     |        |  |  |  |
  |  |     |      /-|   \-|  |   /-DOM/2016/BB_0183
  |  |     |     |  |     |   \-|
  |  |     |     |  |     |      \-DOM/2016/MA_WGS16_011
   \-|   /-|     |  |     |
     |  |  |     |  |      \-DOM/2016/BB_0433
     |  |  |     |  

#### Print the nodes in each branch

The resulting tree is stored in Newick format. Branch lengths in this tree measure nucleotide divergence.

## Get a Time-Resolved Tree
Augur can also adjust branch lengths in this tree to position tips by their sample date and infer the most likely time of their ancestors, using [TreeTime](https://github.com/neherlab/treetime). Run the `refine` command to apply TreeTime to the original phylogenetic tree and produce a “time tree”.

In [94]:
!augur refine \
  --tree zika-tutorial/results/tree_raw.nwk \
  --alignment zika-tutorial/results/aligned.fasta \
  --metadata zika-tutorial/data/metadata.tsv \
  --output-tree zika-tutorial/results/tree.nwk \
  --output-node-data zika-tutorial/results/branch_lengths.json \
  --timetree \
  --coalescent opt \
  --date-confidence \
  --date-inference marginal \
  --clock-filter-iqd 4

pruning leaf  KX369547.1

1.57	###TreeTime.run: INITIAL ROUND

6.34	###TreeTime.run: ITERATION 1 out of 2 iterations

13.54	###TreeTime.run: ITERATION 2 out of 2 iterations

20.64	###TreeTime.run: CONVERGED

41.07	###TreeTime.run: FINAL ROUND - confidence estimation via marginal reconstruction

Inferred a time resolved phylogeny using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

updated tree written to zika-tutorial/results/tree.nwk
node attributes written to zika-tutorial/results/branch_lengths.json


### Documentation for each command
#### `!augur refine`
  `--tree zika-tutorial/results/tree_raw.nwk`
  
  `--alignment zika-tutorial/results/aligned.fasta`
  
  `--metadata zika-tutorial/data/metadata.tsv`
  
  `--output-tree zika-tutorial/results/tree.nwk`
  
  `--output-node-data zika-tutorial/results/branch_lengths.json`: File name to write branch lengths as node data
  
  `--timetree`: Produce timetree using treetime
  
  `--coalescent opt`: coalescent time scale in units of inverse clock rate (float), optimize as scalar ('opt'), or skyline ('skyline')
  
  `--date-confidence`: Calculate confidence intervals for node dates (default: False)
  
  `--date-inference marginal`: {joint,marginal} assign internal nodes to their marginally most likely dates, not jointly most likely (default: joint)
  
  `--clock-filter-iqd 4`: clock-filter: remove tips that deviate more than n_iqd interquartile ranges from the root-to-tip vs time regression (default: None)
      

In [96]:
# "Time resolved" tree
tree = Tree("zika-tutorial/results/tree.nwk", format=1)
print(tree)


      /-Thailand/1610acTw
     |
   /-|   /-SG_018
  |  |  |
  |   \-|   /-SG_056
  |     |  |
  |      \-|--SG_027
  |        |
--|         \-SG_074
  |
  |      /-ZKC2/2016
  |   /-|
  |  |   \-SMGC_1
  |  |
  |  |   /-1_0087_PF
   \-|  |
     |  |   /-1_0181_PF
     |  |  |
     |  |  |--1_0199_PF
      \-|  |
        |  |      /-PRVABC59
        |  |   /-|
        |  |  |  |   /-Brazil/2016/ZBRC16
        |  |  |   \-|
        |  |  |     |   /-V8375
         \-|  |      \-|
           |  |        |   /-HND/2016/HU_ME59
           |  |         \-|
           |  |            \-Nica1_16
           |  |
           |  |   /-Brazil/2015/ZBRC301
           |  |  |
           |  |  |   /-Brazil/2015/ZBRC303
           |  |--|--|
            \-|  |   \-BRA/2016/FC_6706
              |  |
              |  |   /-Colombia/2016/ZC204Se
              |   \-|
              |     |   /-PAN/CDC_259359_V1_V3/2015
              |      \-|
              |        |   /-VEN/UF_1/2016
              |  

In [97]:
from ete3 import TreeNode

strains_raw = []
for node in tree_raw.traverse("postorder"):
        strains_raw.append(node.name)

        
strains_tree = []
for node in tree.traverse("postorder"):
    strains_tree.append(node.name)
# print(strains_raw)
# print(strains_tree)

# For TR-tree in Raw
overlap_TR = []
excluded_TR = []
ignore = 0
for TR in strains_tree:
    for raw in strains_raw:
        if raw == TR:
            overlap_TR.append(TR)
            ignore = 1
    if ignore == 0 and TR not in excluded_raw:
        excluded_TR.append(TR)
    ignore = 0

         
            
# For Raw in TR-tree
overlap_raw = []
excluded_raw = []
ignore = 0
for raw in strains_raw:
    for TR in strains_tree:
        if raw == TR:
            overlap_raw.append(raw)
            ignore = 1
    if ignore == 0 and raw not in excluded_raw:
        excluded_raw.append(raw)
    ignore = 0


print("Raw > TR-tree overlap: ", overlap_raw, "\n\n", "Raw > TR-tree exclusion: ", excluded_raw, "\n")
print("TR-tree > Raw overlap: ", overlap_TR, "\n\n", "TR-tree > Raw exclusion: ", excluded_TR)

# TR-tree has more items because we've inferred additional sequences (NODEs)
# When you run 'augur refine' it says it's pruning KX369547.1 
# so it makes sense that that is the only tip that's in Raw but not in TR
# In addition, if you run 'refine' without the iqd option then KX369547.1 is in both trees


Raw > TR-tree overlap:  ['PAN/CDC_259359_V1_V3/2015', 'COL/FLR_00024/2015', 'COL/FLR_00008/2015', 'VEN/UF_1/2016', 'Colombia/2016/ZC204Se', 'Brazil/2015/ZBRC301', 'Brazil/2015/ZBRA105', 'USA/2016/FL022', 'USA/2016/FLWB042', 'DOM/2016/BB_0059', 'USA/2016/FLUR022', 'Aedes_aegypti/USA/2016/FL05', 'DOM/2016/BB_0183', 'DOM/2016/MA_WGS16_011', 'DOM/2016/BB_0433', 'EcEs062_16', 'HND/2016/HU_ME59', 'V8375', 'Nica1_16', 'Brazil/2016/ZBRC16', 'PRVABC59', 'ZKC2/2016', 'SMGC_1', 'SG_027', 'SG_074', 'SG_056', 'SG_018', 'Thailand/1610acTw', '1_0087_PF', '1_0181_PF', '1_0199_PF', 'Brazil/2015/ZBRC303', 'BRA/2016/FC_6706'] 

 Raw > TR-tree exclusion:  ['', 'KX369547.1'] 

TR-tree > Raw overlap:  ['Thailand/1610acTw', 'SG_018', 'SG_056', 'SG_027', 'SG_074', 'ZKC2/2016', 'SMGC_1', '1_0087_PF', '1_0181_PF', '1_0199_PF', 'PRVABC59', 'Brazil/2016/ZBRC16', 'V8375', 'HND/2016/HU_ME59', 'Nica1_16', 'Brazil/2015/ZBRC301', 'Brazil/2015/ZBRC303', 'BRA/2016/FC_6706', 'Colombia/2016/ZC204Se', 'PAN/CDC_259359_V1_V3

 ### Output sample for branch_lengths.json

In [18]:
with open("zika-tutorial/results/branch_lengths.json") as branch_lengths:
    data = json.load(branch_lengths)

# There's got to be a better way to iterate through just the first few items in this dictionary
header = {}
header.update({'alignment' : data['alignment']})
header.update({'clock' : data['clock']})
header.update({'input_tree' : data['input_tree']})

print(json.dumps(header, indent=4), 
      json.dumps(next(iter(data["nodes"]))), 
      json.dumps(next(iter(data["nodes"].values())), indent=4))


{
    "alignment": "zika-tutorial/results/aligned.fasta",
    "clock": {
        "intercept": -2.2073612841458785,
        "rate": 0.0010969656567080136,
        "rtt_Tmrca": 2012.242836088556
    },
    "input_tree": "zika-tutorial/results/tree_raw.nwk"
} "1_0087_PF" {
    "branch_length": 0.0002899093716128946,
    "clock_length": 0.0002899093716128946,
    "date": "2013-12-10",
    "mutation_length": 0.00027912850217082946,
    "num_date_confidence": [
        2013.9171800136892,
        2013.9993155373031
    ],
    "numdate": 2013.9427569072413,
    "raw_date": "2013-12-XX"
}


In addition to assigning times to internal nodes, the `refine` command filters tips that are likely outliers and assigns confidence intervals to inferred dates. Branch lengths in the resulting Newick tree measure adjusted nucleotide divergence. All other data inferred by TreeTime is stored by strain or internal node name in the corresponding JSON file.

# Annotate the Phylogeny
## Reconstruct Ancestral Traits
TreeTime can also infer ancestral traits from an existing phylogenetic tree and metadata annotating each tip of the tree. The following command infers the **region** and **country** of all internal nodes from the time tree and original strain metadata. As with the `refine` command, the resulting JSON output is indexed by strain or internal node name.

In [19]:
!augur traits \
  --tree zika-tutorial/results/tree.nwk \
  --metadata zika-tutorial/data/metadata.tsv \
  --output zika-tutorial/results/traits.json \
  --columns region country \
  --confidence


Inferred ancestral states of discrete character using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

results written to zika-tutorial/results/traits.json


### Documentation for each command
#### `!augur traits`

`--tree zika-tutorial/results/tree.nwk`

`--metadata zika-tutorial/data/metadata.tsv`

`--output zika-tutorial/results/traits.json`

`--columns region country`: Metadata fields to perform discrete reconstruction on (default: None)

`--confidence`: Record the distribution of subleading mugration states (default: False)

**Subleading mugration states?**



### Output sample for file traits.json

In [20]:
with open("zika-tutorial/results/traits.json") as traits:
    data = json.load(traits)

    
print(json.dumps(next(iter(data['nodes'])), indent=4), 
      json.dumps(next(iter(data['nodes'].values())), indent=4))


"1_0087_PF" {
    "country": "french_polynesia",
    "country_confidence": {
        "french_polynesia": 1.0
    },
    "country_entropy": -1.000088900581841e-12,
    "region": "oceania",
    "region_confidence": {
        "oceania": 1.0
    },
    "region_entropy": -1.000088900581841e-12
}


## Infer Ancestral Sequences
Next, infer the ancestral sequence of each internal node and identify any nucleotide mutations on the branches leading to any node in the tree.

In [21]:
!augur ancestral \
  --tree zika-tutorial/results/tree.nwk \
  --alignment zika-tutorial/results/aligned.fasta \
  --output zika-tutorial/results/nt_muts.json \
  --inference joint

**WARNING: Number of sequences in tree differs from number of sequences in alignment!**

Inferred ancestral sequence states using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

ancestral sequences written to zika-tutorial/results/nt_muts.json


### Documentation for each command
#### `!augur ancestral`

`--tree zika-tutorial/results/tree.nwk`

`--alignment zika-tutorial/results/aligned.fasta`

`--output zika-tutorial/results/nt_muts.json`

`--inference joint`: {joint,marginal} calculate joint or marginal maximum likelihood ancestral sequence states (default: joint)


### Output sample for file nt_muts.json

In [22]:
with open("zika-tutorial/results/nt_muts.json") as nt_muts:
    data = json.load(nt_muts)

# print(json.dumps(next(iter(data['nodes'])), indent=4), json.dumps(data['nodes']['1_0087_PF']['muts'],indent=4))
# for tip in data['nodes]':
#                 print(json.dumps(data['nodes'], indent=4), json.dumps(data['nodes'][tip]['muts'], indent=4))
print(json.dumps(data, indent=4))

{
    "nodes": {
        "1_0087_PF": {
            "muts": [
                "C3433T",
                "A3639G",
                "G8478A"
            ],
            "sequence": "GAATCAGACTGCGACAGTTCGAGTTTGAAGCGAAAGCTAGCAACAGTATCAACAGGTTTTATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAACCCAAAAAAGAAATCCGGAGGATTCCGGATTGTCAATATGCTAAAACGCGGAGTAGCCCGTGTGAGCCCCTTTGGGGGCTTGAAGAGGCTGCCAGCCGGACTTCTGCTGGGTCATGGGCCCATCAGGATGGTCTTGGCGATTCTAGCCTTTTTGAGATTCACGGCAATCAAGCCATCACTGGGTCTCATCAATAGATGGGGTTCAGTGGGGAAAAAAGAGGCTATGGAAATAATAAAGAAGTTCAAGAAAGATCTGGCTGCCATGCTGAGAATAATCAATGCTAGGAAGGAGAAGAAGAGACGAGGCGCAGATACTAGTGTCGGAATTGTTGGCCTCCTGCTGACCACAGCTATGGCAGCGGAGGTCACTAGACGTGGGAGTGCATACTATATGTACTTGGACAGAAACGATGCTGGGGAGGCCATATCTTTTCCAACCACATTGGGGATGAATAAGTGTTATATACAGATCATGGATCTTGGACACATGTGTGATGCCACCATGAGCTATGAATGCCCTATGCTGGATGAGGGGGTGGAACCAGATGACGTCGATTGTTGGTGCAACACGACGTCAACTTGGGTTGTGTACGGAACCTGCCATCACAAAAAAGGTGAAGCACGGAGATCTAGAAGAGCTGTGACGCTCCCCTCCCATTCCACTAGGAAGCTGCAAACGCGGTCGCAAACCTGGTTGGAATCAAGAGAATACACAAAGCACTT

## Identify Amino-Acid Mutations
Identify amino acid mutations from the nucleotide mutations and a reference sequence with gene coordinate annotations. The resulting JSON file contains amino acid mutations indexed by strain or internal node name and by gene name. To export a FASTA file with the complete amino acid translations for each gene from each node’s sequence, specify the `--alignment-output` parameter in the form of `results/aligned_aa_%GENE.fasta`.

In [23]:
!augur translate \
  --tree zika-tutorial/results/tree.nwk \
  --ancestral-sequences zika-tutorial/results/nt_muts.json \
  --reference-sequence zika-tutorial/config/zika_outgroup.gb \
  --output zika-tutorial/results/aa_muts.json

Read in 13 features from reference sequence file
amino acid mutations written to zika-tutorial/results/aa_muts.json


### Documentation for each command
#### `!augur translate`

`--tree zika-tutorial/results/tree.nwk`

`--ancestral-sequences zika-tutorial/results/nt_muts.json`: JSON (fasta input) or VCF (VCF input) containing ancestral and tip sequences (default: None)

`--reference-sequence zika-tutorial/config/zika_outgroup.gb`: GenBank or GFF file containing the annotation (default: None)

`--output zika-tutorial/results/aa_muts.json`


### Output sample for file aa_muts.json

In [3]:
with open("zika-tutorial/results/aa_muts.json") as aa_muts:
    data = json.load(aa_muts)

# This is kind of a crappy example since there are no mutations in the first node
print(json.dumps(next(iter(data['nodes'])), indent=4), 
      json.dumps(next(iter(data['nodes'].values())), indent=4))

# How is it possible that so many strains don't have mutations?
print(json.dumps(data['nodes'], indent=4))

"1_0087_PF" {
    "aa_muts": {
        "2K": [],
        "CA": [],
        "ENV": [],
        "MP": [],
        "NS1": [],
        "NS2A": [],
        "NS2B": [],
        "NS3": [],
        "NS4A": [],
        "NS4B": [],
        "NS5": [],
        "PRO": []
    }
}
{
    "1_0087_PF": {
        "aa_muts": {
            "2K": [],
            "CA": [],
            "ENV": [],
            "MP": [],
            "NS1": [],
            "NS2A": [],
            "NS2B": [],
            "NS3": [],
            "NS4A": [],
            "NS4B": [],
            "NS5": [],
            "PRO": []
        }
    },
    "1_0181_PF": {
        "aa_muts": {
            "2K": [],
            "CA": [],
            "ENV": [],
            "MP": [],
            "NS1": [],
            "NS2A": [],
            "NS2B": [],
            "NS3": [],
            "NS4A": [],
            "NS4B": [],
            "NS5": [],
            "PRO": []
        }
    },
    "1_0199_PF": {
        "aa_muts": {
            "2K": [],
   

**How is it possible that so many sites don't have mutations?**

**Why does only "NODE_0000034" have an aa_sequence?**

In [41]:
# Compare number of aa_muts between two tips
# OR
# compare number of aa_muts between tip and parent
## Will need to figure out how to associate a tip with a NODE
### Maybe ask Sarah about this... I have no idea



from collections import OrderedDict
ordered = OrderedDict(data)
dir(ordered)
# keys = ordered.keys()
# values = ordered.values()
# for k,v in ordered.items():
#     ind = ordered.keys().index(k)
    
#     if k == ch:
#         ind = d.keys().index(k)
#         print keys[ind-1], ':',values[ind-1]
#         print keys[ind+1], ':',values[ind+1]
        

for i,v in enumerate(keyList):
    print(ordered(keyList[i]))
    print(ordered(keyList[i+1]))
        
        
# for node in data:
#     print(data.keys())
#     print(next(iter((data[node].keys()))))
#     previous = data['nodes'][node]['aa_muts']
#     if data['nodes'][node]['aa_muts'] == previous

# print(data['nodes'].keys())


# keyList=(data['nodes'].values())
# for i,previous in enumerate(keyList):
#     previous = data[keyList[i-1]]
#     print(previous)

   
#         print d[keyList[i+1]]
#         print d[keyList[i-1]]


TypeError: 'odict_keys' object does not support indexing

# Below are the instructions to visualize the data with auspice

## Export the Results
Finally, collect all node annotations and metadata and export it all in auspice’s JSON format. This refers to three config files to define colors via `config/colors.tsv`, lat/long coordinates via `config/lat_longs.tsv` and page title, maintainer, filters present, etc… via `config/auspice_config.json`. The resulting tree and metadata JSON files are the inputs to the auspice visualization tool.

Copy the resulting .json files into your auspice/data folder

In [ ]:
!augur export \
  --tree zika-tutorial/results/tree.nwk \
  --metadata zika-tutorial/data/metadata.tsv \
  --node-data zika-tutorial/results/branch_lengths.json \
              zika-tutorial/results/traits.json \
              zika-tutorial/results/nt_muts.json \
              zika-tutorial/results/aa_muts.json \
  --colors zika-tutorial/config/colors.tsv \
  --lat-longs zika-tutorial/config/lat_longs.tsv \
  --auspice-config zika-tutorial/config/auspice_config.json \
  --output-tree auspice/zika_tree.json \
  --output-meta auspice/zika_meta.json

In [ ]:
with open("auspice/zika_meta.json") as meta:
    data = json.load(meta)

# This is kind of a crappy example since there are no mutations in the first node
print(json.dumps(data, indent=4))

## Visualize the Results

I'm not sure how to do this from Jupyter notebook... These commands work if enterred in the command line.

In [ ]:
!cd ~/auspice/data/ 
!npm run dev